# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [ ]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [ ]:
ratings = ratings[ratings['userId'] == 1]
ratings.groupby('rating').count()[['timestamp']].head()

In [ ]:
ratings[ratings.userId == 1].rating.value_counts()

Как это сделать для всех пользователей сразу:

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


In [ ]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

### Упражнение
Какой пользователь выставил больше всех пятерок?

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0,
                    margins = True).sort_values(5.0, ascending=False).head(1)

### Фильтры и вычисления с помощью loc и iloc

In [ ]:
log = pd.read_csv('visit_log(1).csv', sep=';')
log.head()

### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [ ]:
log.url.value_counts().head(5)

In [ ]:
log.url.apply(lambda x: x[16:]).value_counts().head(5)

In [ ]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

In [ ]:
log.loc[log.user_id == 'a8c40697fb', ['user_id', 'region']].head()

In [ ]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

In [ ]:
log.iloc[:, [1, 2]].head()

In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [ ]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

In [ ]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [ ]:
log.loc[(log.traffic_source.isin(['yandex' ,'google'])), 'traffic_type'] = 'organic'
log.head(10)

### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [ ]:
import numpy as np

In [ ]:
None, Not a Number

In [ ]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

In [ ]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [ ]:
log.loc[pd.isnull(log.traffic_type), 'traffic_type'] = 'other'
log

In [ ]:
df.value.mean()

In [ ]:
df.value.fillna(0).mean()

### Дата и время в pandas

In [ ]:
log.head()

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s') # %Y-%m-%d
log.head()

In [ ]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [ ]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

In [ ]:
df['visit_dt'] = pd.to_datetime(df['visit_datetime'])
df['purchase_dt'] = pd.to_datetime(df['purchase_datetime'])
df['time'] = (df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
df.time.mean()

In [ ]:
df

In [ ]:
df['visit_dt'] = pd.to_datetime(df['visit_datetime'])
df['purchase_dt'] = pd.to_datetime(df['purchase_datetime'])
df['delta_dt'] = (df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
df['delta_dt'].mean()

### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

In [ ]:
'книги', 'книги,' , 'книги.'

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [1]:
from pymystem3 import Mystem

In [ ]:
pip install pymystem3

In [2]:
search = 'курс гривны к рублю рубли рублях рубленая'

In [3]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas



['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубленый',
 '\n']

In [4]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль   рубленый \n'

In [5]:
'стали' --> сталь
'стали' --> стать

SyntaxError: invalid syntax (<ipython-input-5-1f48c88473a8>, line 1)

Домашнее задание

Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [7]:
import pandas as pd

In [ ]:
log = pd.read_csv('visit_log(1).csv', sep=';')
log.head()

In [ ]:
log.loc[(log.traffic_source.isin(['yandex' ,'google'])), 'source_type'] = 'organic'
log.loc[(log.traffic_source.isin(['paid' ,'email']) & (log.region == 'Russia')), 'source_type'] = 'ad'
log.loc[(log.traffic_source.isin(['paid' ,'email']) & (log.region != 'Russia')), 'source_type'] = 'other'
log.loc[pd.isnull(log.source_type), 'source_type'] = log.traffic_source
log.head(10)

Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
log = pd.read_csv('URLs.txt')
log.head(10)

In [ ]:
log[log.url.str.contains(pat = '\/\d{8}\-.', regex=True)].head()

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [10]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data.head()

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [14]:
m = Mystem()
data['lemmas'] = data['keyword'].apply(lambda x: [m.lemmatize(x) for y in x])
data.head()

,keyword,shows,lemmas
0,курс гривны к рублю,125076,"[[курс, , гривна, , к, , рубль, \n], [курс,..."
1,доллары в рубли,114173,"[[доллар, , в, , рубль, \n], [доллар, , в, ..."
2,100 долларов в рублях,97534,"[[100, , доллар, , в, , рубль, \n], [100, ..."
3,курс рубля,53546,"[[курс, , рубль, \n], [курс, , рубль, \n], [..."
